In [ ]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [1]:
# Import tensorflow.
import tensorflow as tf

# PIL (Pillow) for working with images.
import PIL
from PIL import Image , ImageOps

# Used for globbing up directories.
import glob

# The Data Tools 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import the Keras tools.
from tensorflow import keras

# Network type.
from tensorflow.keras.models import Sequential

# Layer information.
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# For editing images. 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.utils import to_categorical

print("Tensorflow version:", tf.__version__)
print("Pandas version:", pd.__version__)
print("Keras version:", keras.__version__)
print("Pillow version:",PIL.__version__,"\n")

physical_devices = tf.config.list_physical_devices('GPU') 
print("GPU's:",physical_devices)
for gpu_instance in physical_devices: 
    tf.config.experimental.set_memory_growth(gpu_instance, True)

Tensorflow version: 2.1.0
Pandas version: 1.1.5
Keras version: 2.2.4-tf
Pillow version: 8.1.0 

GPU's: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


## **Image Pipeline**

Set the path the the images. This need to be structured as classes are in directories together. 

In [2]:
src_path_train = "../Data/mnist_images/trainingSet/trainingSet"

The ImageDataGenerator allows us to modify the images to increase the size of the data set. This will be useful if we don't have enough labeled images but we sill want to train the network. 

In [3]:
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
datagen = ImageDataGenerator(
        rescale = 1 / 255.0,
        #rotation_range=20,
        #zoom_range=5,
        #width_shift_range=5,
        #height_shift_range=5,
        #shear_range=100,
        horizontal_flip=True,
        #fill_mode="nearest",
        validation_split=0.20)

In the next cell we are going to setup the data flow pipeline using flow_from_directory. This tool allows us to use Keras to feed the images from the directories into the network. It will also allow us to set a batch size as well as a few other setting for the network. 

In [4]:
batch_size = 100
train_generator = datagen.flow_from_directory(
    directory=src_path_train,
    target_size=(28, 28),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    subset='training',
    shuffle=True,
    seed=42
)

Found 33607 images belonging to 10 classes.


## **Check a few Images**

In [5]:
image_list = []

for images,labels in next(zip(train_generator)):
    for i in range(10): # can't be greater than 20
        image_list.append(images[i])

plt.matshow(image_list[0])
plt.show()

In [57]:
np.shape(image_list)

(10, 28, 28, 1)

## **The Model**

In [127]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(5,5),activation='relu',input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

'''
model.add(Conv2D(128, kernel_size=(5,5),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(512, kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
'''

model.add(Flatten())

model.add(Dense(300, activation='relu'))
#model.add(Dropout(.5))
model.add(Dense(200, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='softmax'))
          
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['accuracy'])

In [128]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 24, 24, 64)        1664      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 300)               2765100   
_________________________________________________________________
dense_5 (Dense)              (None, 200)               60200     
_________________________________________________________________
dense_6 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_7 (Dense)              (None, 10)               

## **Training**

In [129]:
model.fit(train_generator, steps_per_epoch = train_generator.n//train_generator.batch_size, epochs=3)

  ...
    to  
  ['...']
Train for 336 steps
Epoch 1/3
336/336 [==============================] - 6s 19ms/step - loss: 0.3703 - accuracy: 0.8782
Epoch 2/3
336/336 [==============================] - 6s 19ms/step - loss: 0.1212 - accuracy: 0.9620
Epoch 3/3
336/336 [==============================] - 7s 20ms/step - loss: 0.0828 - accuracy: 0.9736
